On Jan 14, 2018 the Authorize.net site was ungoing mainentence, or something, and this quick audit is to make sure that any credit card transactions during that time were processed succesfully.

### Libraries

In [1]:
import sys
sys.path.insert(0, '/Users/jarad/Fake Folder/Python Libraries')

from jb_libraries import *
%matplotlib inline

from osh_data import *
import math

### Script settings

In [23]:
date_start = '2019-01-14'

### Get orders data

In [2]:
pd.read_sql('''select distinct payment_module_code from orders where year(date_purchased) = 2019 order by payment_module_code''', db)

,payment_module_code
0,adabox_subscribe
1,authorizenet_aim
2,authorizenet_cim
3,bitpay
4,GV/DC
5,lapwa
6,paypalwpp
7,purchaseOrder


In [101]:
orders_main = pd.read_sql(
'''
SELECT
orders_id,
order_total,
payment_module_code,
LOWER(orders_status_name) AS orders_status_name,
LOWER(bs_status_name) AS orders_billing_status
FROM orders o
JOIN orders_status os ON o.orders_status = os.orders_status_id
JOIN billing_status bs ON o.orders_billing_status = bs.bs_status_id
WHERE payment_module_code IN ('authorizenet_aim','authorizenet_cim')
AND DATE(date_purchased) = ' '''+ date_start +''' '
AND orders_status NOT IN (8,9,14,15) # fraud pending, fraud confirmed, voided, fraud - void
''', db)

col_fix(orders_main)

print('{:,.0f} orders'.format(len(orders_main)))

551 orders


### Get database credit card data

In [144]:
min_oid = orders_main['orders id'].min()

db1 = pd.read_sql(
'''
SELECT
order_id AS orders_id,
response_code AS db_response_code,
LOWER(authorization_type) AS transaction_type,
transaction_id
FROM authorizenet
WHERE order_id >= '''+ str(min_oid) +'''
''', db)

db2 = pd.read_sql(
'''
SELECT
orders_id,
response_code AS db_response_code,
LOWER(authorization_type) AS transaction_type,
transaction_id
FROM authorizenet_zero
WHERE orders_id >= '''+ str(min_oid) +'''
''', db)

db_main = pd.concat([db1, db2], sort = True)
col_fix(db_main)

response_codes_dict = {1:'approved',
                       2:'declined',
                       3:'error',
                       4:'action required'}
db_main['response code str'] = db_main['db response code'].map(response_codes_dict)

db_main.drop_duplicates(inplace = True)
db_main['transaction id'] = pd.to_numeric(db_main['transaction id'])

db_main['transaction type'] = [x.replace('_',' ') for x in db_main['transaction type']]

d = {'prior auth capture':'capture',
    'auth capture':'capture'}

for k,v in d.items():
    db_main['transaction type'] = db_main['transaction type'].replace(k,v)
    
db_main['orders id'] = db_main['orders id'].map(int)    

db_main.dropna(subset = ['transaction id'], inplace = True)
db_main['transaction id'] = db_main['transaction id'].map(int)

### Map transaction id

In [160]:
to_map = db_main[db_main['transaction type'] == 'capture']

orders_main['transaction id'] = orders_main['orders id'].map(dict(zip(to_map['orders id'], to_map['transaction id'])))
orders_main['transaction id'] = orders_main['transaction id'].map(int)

n = orders_main[orders_main.isnull().any(1)]
if n.empty == False:
    raise ValueError('check ur nulls')

### Get CSV data

In [161]:
path = '/Users/jarad/Fake Folder/Finance/Ad Hoc/Authorize.net Transaction Check/Docs/For Jan 14, 2019 Auth net check.txt'

csv_main = pd.read_csv(path)

csv_main.columns = csv_main.columns.str.lower()

for col in csv_main.columns:
    try:
        csv_main[col] = csv_main[col].str.lower()
    except:
        pass

### Map CSV data

In [162]:
ls = ['transaction status',
     'settlement amount']

for x in ls:
    orders_main[x] = orders_main['transaction id'].map(dict(zip(csv_main['transaction id'], csv_main[x])))
    
n = orders_main[orders_main.isnull().any(1)]

if n.empty == False:
    raise ValueError('check ur nulls')    

### Check transaction statuses

In [163]:
s = set(orders_main['transaction status'])

for x in s:
    if x != 'settled successfully':
        raise ValueError('check ur statuses')

### Check mismatches

In [164]:
orders_main['difference'] = np.abs(orders_main['order total'] - orders_main['settlement amount'])

s = set(orders_main[orders_main['difference'] > 0]['orders billing status'])

for x in s:
    if x != 'refunded':
        raise ValueError('check ur statuses')
    else:
        print('all mismatches stem from refunds')

all mismatches stem from refunds


### View all auths with no captures
Note that this data is Jan 14, 2019 and onwards; the actual day in question (Jan 14, 2019) has been already reconciled, up above

In [165]:
by_oid = db_main.groupby(['orders id','transaction type'])[['transaction type']].count().unstack(1)
by_oid.columns = by_oid.columns.droplevel(0)
by_oid.dropna(how = 'all', axis = 1, inplace = True)

In [173]:
no_captures = by_oid[by_oid.iloc[:,1:].isnull().any(1)]

o = pd.read_sql(
'''
SELECT
orders_id,
LOWER(payment_method) AS payment_method,
LOWER(orders_status_name) AS orders_status_name
FROM orders o
JOIN orders_status os ON o.orders_status = os.orders_status_id
WHERE orders_id IN '''+ str(tuple(no_captures.index.tolist())) +'''
''', db)

col_fix(o)

In [174]:
o[o['payment method'] == 'credit card']['orders status name'].value_counts()

processing         224
printed            185
voided               9
fraud - pending      6
fraud - void         3
Name: orders status name, dtype: int64